In [19]:
import os
import whisper
import polars as pl
import pickle
from babel.dates import format_time

In [24]:
# Function to transcribe audio using OpenAI's transcription service
def transcribe_audio(model, audio_path, transcript_path, df):
    n = df.shape[0]
    temp_column = []
    for i in range(0, n):
        print("Processing segment " + str(i) + " of " + str(n))
        file_path = audio_path + "/" + df[i, 'file_id'] + "_" + str(round(df[i, 'start'], 3)) + ".wav"
        result = model.transcribe(file_path)
        temp_column.append(result['text']) # add transcript text to column to add to df
        with open(transcript_path + "/" + df[i, 'file_id'] + "_" + str(round(df[i, 'start'], 3)) + ".pkl", "wb") as fp:   # pickling full transcript data
            pickle.dump(result, fp)
        print("Segment " + str(i) + " of " + str(n) + " complete")
    df = df.with_columns(pl.Series(name="transcript", values=temp_column))
    return(df)


In [14]:
df = pl.read_csv("../data/ad_validation_initial.csv")
df_test = df[6:8]
print(df_test)

shape: (2, 7)
┌──────────────────────────────┬─────────────┬────────┬──────────┬───────────┬─────────────┬───────┐
│ file_id                      ┆ speaker_est ┆ start  ┆ duration ┆ end       ┆ speaker_val ┆ notes │
│ ---                          ┆ ---         ┆ ---    ┆ ---      ┆ ---       ┆ ---         ┆ ---   │
│ str                          ┆ str         ┆ f64    ┆ f64      ┆ f64       ┆ str         ┆ str   │
╞══════════════════════════════╪═════════════╪════════╪══════════╪═══════════╪═════════════╪═══════╡
│ pres_trimmed_incl_scene-P-19 ┆ SPEAKER_03  ┆ 33.998 ┆ 8.472    ┆ 42.47     ┆ other       ┆ null  │
│ 04-1…                        ┆             ┆        ┆          ┆           ┆             ┆       │
│ pres_trimmed_incl_scene-P-19 ┆ SPEAKER_02  ┆ 42.47  ┆ 2.088    ┆ 44.558002 ┆ biden       ┆ null  │
│ 04-1…                        ┆             ┆        ┆          ┆           ┆             ┆       │
└──────────────────────────────┴─────────────┴────────┴──────────┴───────────

In [25]:
model = whisper.load_model("large")
df_trans = transcribe_audio(model, "../data/ad_audio_split_testing", "../data/ad_transcripts_testing", df_test)

Processing segment 0 of 2


/Users/dominicvalentino/Documents/Harvard/presidential_recognition/pr/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Segment 0 of 2 complete
Processing segment 1 of 2


/Users/dominicvalentino/Documents/Harvard/presidential_recognition/pr/lib/python3.9/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Segment 1 of 2 complete
shape: (2, 8)
┌───────────────┬─────────────┬────────┬──────────┬───────────┬─────────────┬───────┬──────────────┐
│ file_id       ┆ speaker_est ┆ start  ┆ duration ┆ end       ┆ speaker_val ┆ notes ┆ transcript   │
│ ---           ┆ ---         ┆ ---    ┆ ---      ┆ ---       ┆ ---         ┆ ---   ┆ ---          │
│ str           ┆ str         ┆ f64    ┆ f64      ┆ f64       ┆ str         ┆ str   ┆ str          │
╞═══════════════╪═════════════╪════════╪══════════╪═══════════╪═════════════╪═══════╪══════════════╡
│ pres_trimmed_ ┆ SPEAKER_03  ┆ 33.998 ┆ 8.472    ┆ 42.47     ┆ other       ┆ null  ┆ You were     │
│ incl_scene-P- ┆             ┆        ┆          ┆           ┆             ┆       ┆ asked, is he │
│ 1904-1…       ┆             ┆        ┆          ┆           ┆             ┆       ┆ ready? Yo…   │
│ pres_trimmed_ ┆ SPEAKER_02  ┆ 42.47  ┆ 2.088    ┆ 44.558002 ┆ biden       ┆ null  ┆ I think that │
│ incl_scene-P- ┆             ┆        ┆          ┆  

In [28]:
print(df_trans['file_id'][0])
print(df_trans['transcript'][0])

pres_trimmed_incl_scene-P-1904-108497
 You were asked, is he ready? You said, I think he can be ready, but right now I don't believe he is. The presidency is not something that lends itself to on-the-job training.
